In [1]:
! python --version
# Should be 3.7

Python 3.7.4


In [2]:
import pandas as pd
import numpy as np
import os
import interpret

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from matplotlib import pyplot

Check working directory is Stagedaniel

In [4]:
cd ..


C:\Users\danie\Documents\StageDaniel\research


In [5]:
cd research

[WinError 2] Het systeem kan het opgegeven bestand niet vinden: 'research'
C:\Users\danie\Documents\StageDaniel\research


In [6]:
os.getcwd()

'C:\\Users\\danie\\Documents\\StageDaniel\\research'

In [8]:
data_name = 'clean_norm_poland'
data_dir = os.getcwd() + '/data/'
data_csv_file = data_dir + data_name + '_processed.csv'

# load data file from csv
df = pd.read_csv(data_csv_file, sep = ',')
data = df.as_matrix()
data_headers = list(df.columns.values)
N = data.shape[0]

# setup Y vector and Y_name
Y_col_idx = [0]
Y = data[:, Y_col_idx]
Y_name = [data_headers[j] for j in Y_col_idx]
Y[Y == 0] = -1

# setup X and X_names
X_col_idx = [j for j in range(data.shape[1]) if j not in Y_col_idx]
X = df.drop(df.columns[0], axis=1)
X_names = [data_headers[j] for j in X_col_idx]

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [9]:
%timeit
ebm = ExplainableBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=0).fit(X, Y)
ebm.score(X, Y)

0.9566017820761432

In [10]:
roc_auc_score(Y,ebm.predict_proba(X)[:,1])

0.8894390624815773

In [11]:
accuracy_score(Y, ebm.predict(X))

0.9566017820761432

In [12]:
print(classification_report(Y, ebm.predict(X)))

              precision    recall  f1-score   support

        -1.0       0.96      1.00      0.98     38807
         1.0       0.91      0.09      0.17      1932

    accuracy                           0.96     40739
   macro avg       0.93      0.55      0.57     40739
weighted avg       0.95      0.96      0.94     40739



In [13]:
from interpret import show
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7365/2458347366600/ -->

In [14]:
ebm_local = ebm.explain_local(X.iloc[:5,:], Y[:5], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7365/2458347365576/ -->

In [10]:
(ebm.score(X, Y)-(1-(Y+1).mean()/2))*N

2577.999999999999

In [11]:
X.iloc[:5,:]

,Age_leq_21,Age_22_to_29,Age_30_to_44,Age_45_to_59,Age_geq_60,EmployedInPrivateSector,EmployedInPublicSector,SelfEmployedNotInc,SelfEmployedInc,HSDiploma,...,WorkHrsPerWeek_geq_50,Male,Female,White,Black,OtherRace,NativeUSorCanada,NativeImmigrant,AnyCapitalGains,AnyCapitalLoss
0,1,0,1,0,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,1,0
1,1,0,0,1,0,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
2,1,0,1,0,0,1,0,0,0,1,...,0,1,0,1,0,0,1,0,0,0
3,1,0,0,1,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0


In [30]:
1-(Y+1).mean()/2

0.9525761555266452